In [7]:
import tensorflow as tf
import numpy as np

# MNIST

In [8]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/mnist/", one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


### mnist 데이터 개수 보는 방법

* 1 : len(mnist.train.labels)
* 2 : mnist.train.num_examples

In [9]:
print len(mnist.train.labels)
print mnist.train.num_examples

55000
55000


In [10]:
print len(mnist.test.labels)
print mnist.test.num_examples

10000
10000


In [11]:
print "training set 개수 : {}".format(len(mnist.train.labels))
print "validation set 개수 : {}".format(len(mnist.validation.labels))
print "test set 개수 : {}".format(len(mnist.test.labels))

training set 개수 : 55000
validation set 개수 : 5000
test set 개수 : 10000


In [12]:
mnist.test.labels[0:5, :]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]])

In [13]:
mnist.test.cls = np.argmax(mnist.test.labels, axis=1)

In [14]:
mnist.test.cls[0:5]

array([7, 2, 1, 0, 4])

### 모델 만들기

* Input layer : 784
* Hidden layer 1 : 256
* Hidden layer 2 : 256
* Output layer : 10

In [15]:
x_true = tf.placeholder(tf.float32, [None,784])
y_true = tf.placeholder(tf.float32, [None,10])

In [16]:
weight_1 = tf.Variable(tf.truncated_normal([784,256], stddev=0.01))
weight_2 = tf.Variable(tf.truncated_normal([256,256], stddev=0.01))
weight_3 = tf.Variable(tf.truncated_normal([256,10], stddev=0.01))

In [17]:
bias_1 = tf.Variable(tf.zeros([256]))
bias_2 = tf.Variable(tf.zeros([256]))
bias_3 = tf.Variable(tf.zeros([10]))

In [18]:
hidden_1 = tf.nn.relu(tf.add(tf.matmul(x_true,weight_1), bias_1))
hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1,weight_2), bias_2))
y_pred = tf.nn.relu(tf.add(tf.matmul(hidden_2, weight_3), bias_3))

In [19]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

### 학습시키기

In [20]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [21]:
batch_size = 100
total_batch = int(mnist.train.num_examples/batch_size)

In [22]:
for epoch in range(10):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, {x_true:batch_xs, y_true:batch_ys})
        total_cost += sess.run(cost, {x_true:batch_xs, y_true:batch_ys})
        
    print "Epoch : {%04d}" % (epoch + 1), "Cost : {:.3f}".format(total_cost / total_batch)
    
print "Optimization completed."

Epoch : {0001} Cost : 1.455
Epoch : {0002} Cost : 1.218
Epoch : {0003} Cost : 1.004
Epoch : {0004} Cost : 0.986
Epoch : {0005} Cost : 0.973
Epoch : {0006} Cost : 0.966
Epoch : {0007} Cost : 0.963
Epoch : {0008} Cost : 0.959
Epoch : {0009} Cost : 0.953
Epoch : {0010} Cost : 0.953
Optimization completed.


### 평가하기

In [23]:
correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print "정확도 :", sess.run(accuracy, {x_true: mnist.test.images,
                                     y_true: mnist.test.labels})

정확도 : 0.5856
